# 加载多通道/多位置的tif文件夹
一些显微镜会将图像数据逐片写入磁盘作为tif文件。你会在这些文件夹中找到许多名为`image_z03_ch01.tif`之类的文件。要加载这些文件夹，你通常需要知道一些细节，比如有多少Z切片和多少通道被成像，或者图像数据是否有多个时间点。

另请参阅[这个讨论](https://forum.image.sc/t/reading-2d-images-as-a-multi-channel-3d-stack-in-python/75460)。

In [1]:
from skimage import io
import matplotlib.pyplot as plt
import tifffile as tif
import numpy as np

import shutil
import os

path = "../../data/tif_folder/"

为了演示目的，我们只是创建这样一个包含虚拟图像数据的文件夹。

In [2]:
if not os.path.exists(path):
    os.mkdir(path)

In [3]:
for z in range(1,7):
    for c in range(1,5):
        #image = io.imread("c:/structure/data/blobs.tif")
        #io.imsave(f"c:/structure/data/images/r01c01f34p0{z}-ch0{c}t01.tiff", image)
        
        shutil.copy("../../data/blobs.tif",
                    path + f"image_z{str(z).zfill(2)}-ch{str(c).zfill(2)}.tiff")

为了获得概览，我们可以打印出文件夹中的文件名。

In [4]:
for file in os.listdir(path):
    print(file)

image_z01-ch01.tiff
image_z01-ch02.tiff
image_z01-ch03.tiff
image_z01-ch04.tiff
image_z02-ch01.tiff
image_z02-ch02.tiff
image_z02-ch03.tiff
image_z02-ch04.tiff
image_z03-ch01.tiff
image_z03-ch02.tiff
image_z03-ch03.tiff
image_z03-ch04.tiff
image_z04-ch01.tiff
image_z04-ch02.tiff
image_z04-ch03.tiff
image_z04-ch04.tiff
image_z05-ch01.tiff
image_z05-ch02.tiff
image_z05-ch03.tiff
image_z05-ch04.tiff
image_z06-ch01.tiff
image_z06-ch02.tiff
image_z06-ch03.tiff
image_z06-ch04.tiff


scikit-image提供了一个[`imread_collection`](https://scikit-image.org/docs/stable/api/skimage.io.html#skimage.io.imread_collection)用于加载匹配某种模式的文件，例如包含`*`的文件。

In [5]:
im_collection = io.imread_collection(path + "*")
im_collection

你可以将这个图像集合转换为基于numpy数组的图像堆栈。不幸的是，此时Z切片和通道的数量是未知的。

In [6]:
image_stack = im_collection.concatenate()
image_stack.shape

(24, 254, 256)

如果你知道Z切片和通道的数量，你可以使用[`reshape`]()将图像重塑为3D+通道或4D图像。

In [7]:
num_channels = 4
num_z_slices = 6
image4d = np.reshape(image_stack, (num_channels, num_z_slices, image_stack.shape[-2], image_stack.shape[-1]))
image4d.shape

(4, 6, 254, 256)

或者，你也可以构建自己的for循环来从磁盘加载数据。这给你提供了更多的自由，例如将切片和通道排序到使用的维度中。

In [8]:
num_channels = 4
num_z_slices = 6

image4d_loaded = np.asarray([
    [io.imread(path + f"image_z{str(z).zfill(2)}-ch{str(c).zfill(2)}.tiff") for c in range(1, 5)]
    for z in range(1, 7)
])
image4d_loaded.shape

(6, 4, 254, 256)